In [1]:
from pyspark.sql.functions import expr, lit, stddev, when, min as min_ps, max as max_ps, sum, mean, col
from pyspark.sql.types import StringType, FloatType, StructType, StructField
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession \
    .builder \
    .appName("Carga") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/08 21:26:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_read = spark.read.format('csv').option("inferSchema","true").option("header", True).load('/home/jovyan/work/MaisTodos/')

# Objetivo  


# 1 - Exploração:

1.1 - Qual a coluna com maior desvio padrão?

In [4]:
df_stddev = df_read.select(stddev("housing_median_age").alias("housing_median_age"),
               stddev("total_rooms").alias("total_rooms"),
               stddev("total_bedrooms").alias("total_bedrooms"),
               stddev("population").alias("population"),
               stddev("households").alias("households"),
               stddev("median_income").alias("median_income"),
               stddev("median_house_value").alias("median_house_value"))
df_stddev.show()

+------------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+
|housing_median_age|      total_rooms|   total_bedrooms|       population|       households|     median_income|median_house_value|
+------------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+
|12.586936981660406|2179.947071452777|421.4994515798648|1147.852959159527|384.5208408559016|1.9081565183791036|115983.76438720895|
+------------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+



In [5]:
std_values = df_stddev.first().asDict()
f"coluna maior desvio padrão: {max(std_values, key=std_values.get)}" 

'coluna maior desvio padrão: median_house_value'

In [6]:
f"coluna maior desvio padrão: {df_stddev.toPandas().loc[0].T.idxmax()}"

'coluna maior desvio padrão: median_house_value'

1.2 - Qual valor mínimo e o máximo?

In [7]:
df_min = df_read.select(min_ps("housing_median_age").alias("housing_median_age"),
                        min_ps("total_rooms").alias("total_rooms"),
                        min_ps("total_bedrooms").alias("total_bedrooms"),
                        min_ps("population").alias("population"),
                        min_ps("households").alias("households"),
                        min_ps("median_income").alias("median_income"),
                        min_ps("median_house_value").alias("median_house_value"))
df_min.show()

+------------------+-----------+--------------+----------+----------+-------------+------------------+
|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+------------------+-----------+--------------+----------+----------+-------------+------------------+
|               1.0|        2.0|           1.0|       3.0|       1.0|       0.4999|           14999.0|
+------------------+-----------+--------------+----------+----------+-------------+------------------+



In [8]:
df_max = df_read.select(max_ps("housing_median_age").alias("housing_median_age"),
                        max_ps("total_rooms").alias("total_rooms"),
                        max_ps("total_bedrooms").alias("total_bedrooms"),
                        max_ps("population").alias("population"),
                        max_ps("households").alias("households"),
                        max_ps("median_income").alias("median_income"),
                        max_ps("median_house_value").alias("median_house_value"))
df_max.show()

+------------------+-----------+--------------+----------+----------+-------------+------------------+
|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+------------------+-----------+--------------+----------+----------+-------------+------------------+
|              52.0|    37937.0|        6445.0|   35682.0|    6082.0|      15.0001|          500001.0|
+------------------+-----------+--------------+----------+----------+-------------+------------------+



In [9]:
df_min.toPandas().loc[0].T

housing_median_age        1.0000
total_rooms               2.0000
total_bedrooms            1.0000
population                3.0000
households                1.0000
median_income             0.4999
median_house_value    14999.0000
Name: 0, dtype: float64

In [10]:
min_values = df_min.first().asDict()
max_values = df_max.first().asDict()

print(f"""valor minimo: {min(min_values, key=std_values.get)}
valor maximo: {max(max_values, key=std_values.get)}""")

valor minimo: median_income
valor maximo: median_house_value


In [11]:
print(f"""valor minimo: {df_min.toPandas().loc[0].T.min()}
valor maximo: {df_max.toPandas().loc[0].T.max()}""")

valor minimo: 0.4999
valor maximo: 500001.0


## 2 - Trabalhando com colunas:

2.1 - Criar coluna hma_cat, baseada na coluna housing_median_age, conforme as regras abaixo:

    *  Se < 18 então de_0_ate_18.
    *  Se >= 18 E < 29 entao ate_29.
    *  Se >= 29 E < 37 entao ate_37.
    *  Se >= 37 então acima_37.  

In [12]:
df_read = df_read.withColumn("hma_cat", when(df_read["housing_median_age"] < 18, "de_0_ate_18")
                                       .when(df_read["housing_median_age"].between(18, 28), "ate_29")
                                       .when(df_read["housing_median_age"].between(29, 36), "ate_37")
                                       .when(df_read["housing_median_age"] >=37, "acima_37")
                                       .otherwise(df_read["housing_median_age"]))

In [13]:
df_read.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+-----------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|    hma_cat|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+-----------+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|de_0_ate_18|
|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|     ate_29|
|  -114.56|   33.69|              17.0|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|de_0_ate_18|
|  -114.57|   33.64|              14.0|     1501.0|         337.0|     515.0|     226.0|       3.1917|           73400.0|de_0_ate_18|
|  -114.57|   33.57|              20.0|     1454.0|         32

2.2 - Criar a coluna c_ns:

    * Onde longitude abaixo (<) de -119 recebe o valor norte e acima(>=) sul. 

In [14]:
df_read = df_read.withColumn("c_ns", when(df_read["longitude"] < -119, "norte")
                                    .otherwise("sul"))

In [15]:
df_read.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+-----------+----+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|    hma_cat|c_ns|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+-----------+----+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|de_0_ate_18| sul|
|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|     ate_29| sul|
|  -114.56|   33.69|              17.0|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|de_0_ate_18| sul|
|  -114.57|   33.64|              14.0|     1501.0|         337.0|     515.0|     226.0|       3.1917|           73400.0|de_0_ate_18| sul|
|  -114.57|   33.57|       

2.3 - Renomer as colunas:

    * hma_cat > age
    * c_ns > california_region

In [16]:
df_read = df_read.withColumnRenamed("hma_cat", "age")\
                 .withColumnRenamed("c_ns", "california_region")

In [17]:
df_read.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+-----------+-----------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|        age|california_region|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+-----------+-----------------+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|de_0_ate_18|              sul|
|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|     ate_29|              sul|
|  -114.56|   33.69|              17.0|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|de_0_ate_18|              sul|
|  -114.57|   33.64|              14.0|     1501.0|         337.0|     515.0|     226.0|

### Escrevendo o resultado localmente em parquet, armazenar os dados no seguinte formato:


| Coluna              | Datatype    |
| --------------------| ----------- |
| `age`               | `string`    |
| `california_region` | `string`    |
| `total_rooms`       | `double`    |
| `total_bedrooms`    | `double`    |
| `population`        | `double`    |
| `households`        | `double`    |
| `median_house_value`| `double`    |


In [18]:
columns = ["age","california_region","total_rooms","total_bedrooms","population","households","median_house_value"]

In [19]:
check_schema = StructType([StructField("age", StringType(), True),
                           StructField("california_region", StringType(), True),
                           StructField("total_rooms", FloatType(), True),
                           StructField("total_bedrooms", FloatType(), True),
                           StructField("population", FloatType(), True),
                           StructField("households", FloatType(), True),
                           StructField("median_house_value", FloatType(), True)])

In [20]:
df_read.select(*columns).write.option("schema",check_schema).format("parquet").mode('overwrite').save("check")

## 3 - Agregações:

3.1 - Escreva um arquivo no formato Parquet localmente considerando o dataframe final, crie a seguinte analise:

    * Age
    * California_region
    * S_population: Soma de population
    * M_median_house_value: Média de median_house_value

In [21]:
final_df = df_read.groupBy(["california_region", "age"])\
                  .agg(sum('population').alias("S_population"), 
                       mean("median_house_value").alias("M_median_house_value"))\
                  .orderBy(col("M_median_house_value").desc())
final_df.show()

+-----------------+-----------+------------+--------------------+
|california_region|        age|S_population|M_median_house_value|
+-----------------+-----------+------------+--------------------+
|              sul|   acima_37|   2519076.0|   227694.8277657267|
|              sul|     ate_29|   3905630.0|  220571.65846153846|
|            norte|   acima_37|   2114160.0|  217732.95624136343|
|              sul|     ate_37|   3435282.0|  212266.99328593997|
|              sul|de_0_ate_18|   4157987.0|  209407.56504269212|
|            norte|     ate_37|   1792950.0|   195766.6032154341|
|            norte|     ate_29|   3107411.0|  188724.02380952382|
|            norte|de_0_ate_18|   3270261.0|  177826.69768637532|
+-----------------+-----------+------------+--------------------+



In [22]:
final_schema = StructType([StructField("age", StringType(), True),
                           StructField("california_region", StringType(), True),
                           StructField("S_population", FloatType(), True),
                           StructField("M_median_house_value", FloatType(), True)])

In [23]:
final_df.write.option("schema", final_schema).format("parquet").mode('overwrite').save("final")